In [4]:
import os
import sys
import psutil
import traci
import time

# Ensure SUMO_HOME is set
if "SUMO_HOME" in os.environ:
    sys.path.append(os.path.join(os.environ["SUMO_HOME"], "tools"))

# Function to kill old SUMO processes before starting
def kill_sumo():
    for process in psutil.process_iter(attrs=["pid", "name"]):
        if "sumo" in process.info["name"].lower():
            print(f"Killing SUMO process {process.info['name']} (PID: {process.info['pid']})")
            os.kill(process.info["pid"], 9)

kill_sumo()

# Run simulation
sumoBinary = "sumo/sumo/bin/sumo"
sumoCmd = [sumoBinary, "-c", "osm.sumocfg"]

try:
    # Increase retries to prevent connection issues
    traci.start(sumoCmd, numRetries=20)

    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        for veh_id in traci.vehicle.getIDList():
            battery_level = traci.vehicle.getParameter(veh_id, "device.battery.energyStored")
            print(f"Vehicle {veh_id} Battery Level: {battery_level} kWh")

except traci.exceptions.FatalTraCIError as e:
    print(f"TraCI connection lost: {e}")

except BrokenPipeError:
    print("BrokenPipeError: SUMO crashed or closed unexpectedly.")

finally:
    time.sleep(1)
    try:
        if traci.connection._connections:
            print("Closing TraCI connection properly...")
            traci.close()
    except Exception as e:
        print(f"TraCI was already closed: {e}")


Killing SUMO process sumo (PID: 6761)
 Retrying in 1 seconds
***Starting server on port 56053 ***


Loading net-file from 'osm.net.xml.gz' ... done (64ms).
Loading additional-files from 'osm.poly.xml.gz' ... done (20ms).
Loading additional-files from 'charging-stations.add.xml' ... done (1ms).
Loading route-files incrementally from 'osm.passenger.trips.xml'
Loading route-files incrementally from 'ev-routes.rou.xml'
Loading done.
The battery device is active for vType 'electric_car' but no emission class is set. The emission class Energy/unknown will be used, please consider setting an explicit emission class!
Simulation version v1_21_0+2163-fe398d3a3bc started with time: 0.00.


Error: Answered with error to command 0xa4: Vehicle 'EV_1' does not support device parameter 'device.battery.energyStored' (Parameter 'energyStored' is not supported for device of type 'battery').


Closing TraCI connection properly...
Simulation ended at time: 1.00.
Reason: TraCI requested termination.
Performance:
 Duration: 1.10s
 TraCI-Duration: 0.00s
 Real time factor: 0.909091
 UPS: 1.818182
Vehicles:
 Inserted: 2 (Loaded: 3)
 Running: 2
 Waiting: 0
Statistics (avg of 0):
 RouteLength: 0.00
 Speed: 0.00
 Duration: 0.00
 WaitingTime: 0.00
 TimeLoss: 0.00
 DepartDelay: 0.00
DijkstraRouter answered 1 queries and explored 40.00 edges on average.
DijkstraRouter spent 0.00s answering queries (0.00ms on average).


TraCIException: Vehicle 'EV_1' does not support device parameter 'device.battery.energyStored' (Parameter 'energyStored' is not supported for device of type 'battery').

In [46]:
traci.close()

BrokenPipeError: [Errno 32] Broken pipe